In [7]:
#PARA QUE EL USUARIO IMPORTE LAS LIBRERÍAS
import sys
import subprocess
import pkg_resources

required  = {'numpy', 'pandas'} 
installed = {pkg.key for pkg in pkg_resources.working_set}
missing   = required - installed

if missing:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *missing])

In [8]:
# LIBRERÍAS UTILIZADAS:
import pandas as pd #para la utilización de dataframe
import numpy as np # para crear array
from base64 import encode #para el encoding de los archivos import encodings
from venv import create#esto creo que no es necesario
from sqlalchemy import create_engine #conexión a MySQL
from IPython.display import HTML, display_html, display#para agregar títulos
from tkinter import Frame #para detectar que el tipo de dato es un dataframe de py
from unicodedata import normalize#para normalizar strings
import re #para normalizar incluyendo la ñ
import os #para desglozar la ruta del archivo
import datefinder #identificar datos tipo objeto/string que son fecha
import datetime #libreria de fechas

#### Pasamos los archivos de diversas fuentes a un df

In [13]:
#desde un csv
ruta= r'C:\Users\Gise\Desktop\Data Since\LABS\Individuales\Datasets relevamiento precios\sucursal.csv'
sucursal=pd.read_csv(ruta, sep=",", encoding="utf-8")
sucursal.head()

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado


ignore_index = True

In [14]:
#desde parquet
ruta_parquet=r'C:\Users\Gise\Desktop\Data Since\LABS\Individuales\Datasets relevamiento precios\producto.parquet'
producto= pd.read_parquet(ruta_parquet,engine='auto')
producto.head()

,id,marca,nombre,presentacion,categoria1,categoria2,categoria3
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un,None,None,None
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima 1 Un,1.0 un,None,None,None
2,0000000205870,SIN MARCA,Ojo de Bife 1 Kg,1.0 kg,None,None,None
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito 1 Kg,1.0 kg,None,None,None
4,0000000205955,SIN MARCA,Chiquizuela Novillito 1 Kg,1.0 kg,None,None,None


En total tenemos 5 archivos, por lo que tenemos que generar 3 tablas:

* sucursal

* precios semana 

* producto


In [9]:
def Estado_general_tabla (df=Frame):

    #Vemos valores null
    nulos=df.isnull().sum().sum()
    #Vemos duplicados y los eliminamos
    duplicados=df.duplicated().sum().sum()
    df.drop_duplicates()

    indice=0
    nombre_columna=[]
    tipo_de_dato=[]
    valores_null=[]
    cantidad_repetidos=[]
    cantidad_valores_unicos=[]
    valores_unicos=[]
    for c in df.columns:
        #vemos si la columna posee un volúmen de datos que sea analizable:
        if df[c].isnull().sum()/(df[c].isnull().sum()+df[c].notnull().sum())>0.7:
            print("Se ha borrado la columna: ",c, "Por falta de datos")
            df.drop(c,axis=1,inplace=True)
        else:
            #Agregamos el nombre de la columna a su lista correspondiente:
            nombre_columna.append(c)
            #detectamos el tipo de dato y lo agregamos a su lista correspondiente:
            tipo_dato=df[c].dtype
            tipo_de_dato.append(tipo_dato)
            
            #Buscamos null y los agregamos a su lista correspondiente:
            nulos=df[c].isnull().sum().sum()
            valores_null.append(nulos)
                            
            #Detectamos las columnas que son string 
            if df[c].dtype == 'object':
                #llenamos valores null
                df[c].fillna("vacio", inplace = True) 
                #ponemos todo mayúsculas en primer palabra y el resto en minúsculas
                df[c]=df[c].str.title() 
                #sacamos los acentos
                for i in df.index: 
                    # -> NFD y eliminar diacríticos
                    s=df[c][i]
                    s = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", s), 0, re.I)
                    # -> NFC
                    s = normalize( 'NFC', s)
                    df[c][i]=s
                            
            # Buscamos los valores repetidos 
            repetidos=df[c].duplicated().sum()
            #agregamos a la lista de repetidos
            cantidad_repetidos.append(repetidos)
            
            #Buscamos valores diferentes (únicos) que existen:
            array_unicos=df[c].unique()
            #los pasamos a lista para una mejor manipulación
            lista_unicos=array_unicos.ravel().tolist()
            
            #agregamos a la lista cantidad de valores únicos 
            cantidad_valores_unicos.append(len(lista_unicos))
            
            #Agregamos a la lista los valores únicos 
            valores_unicos.append(df[c].unique())
                        
            #Sumamos uno al índice para que continúe recorriendo las columnas
            indice+=1
        
    df_resultado=pd.DataFrame()
    df.style.set_caption('Mi Tabla')
    df_resultado["Nombre_columna"]=nombre_columna
    df_resultado["Tipo_de_dato"]=tipo_dato
    df_resultado["Cantidad_null"]=valores_null
    df_resultado['Cantidad_repetidos']=cantidad_repetidos
    df_resultado["cantidad_valores_unicos"]=cantidad_valores_unicos
    df_resultado["valores_unicos"]=valores_unicos
    #display(df_resultado.style.set_caption('Mi Tabla resúmen'));

In [10]:
def limpieza_idsucursal (df):
    #separo los id que tienen - y las guardo en una variable
    ver_id=df["id"].str.split('-', expand=True)
    #renombro las columnas para poder usarlas
    ver_id.columns=["comercioid",'banderaid',"idSucursal"]
    df["id"]=ver_id['idSucursal']
    #renombro la columna id
    df.rename(columns={'id': 'idSucursal'}, inplace=True)
    df['idSucursal'] = df['idSucursal'].astype('int32')
    return df

In [11]:
def limpieza_producto (df):
    #LIMPIEZA ID PRODUCTO
    #le cambio el nombre de la columna para que sea descriptiva
    df.rename(columns={'id': 'idProducto'}, inplace=True)
    df=limpieza_id_producto(df)
    #LIMPIEZA NOMBRE DE PRODUCTO
    df.rename(columns={'nombre': 'nombreProducto'}, inplace=True)
    df['nombreProducto']=df['nombreProducto'].apply(lambda x :(" ".join((x.split())[:-2])) if x !=0 else x )
    #papa 1kg--> papa
    return df
    

In [19]:
def limpieza_id_producto (df):
    #reemplazo los valores con -
    df.idProducto.replace("-",regex=True) 

    #Saco los espacios si existen en la columna idProducto
    df['idProducto']=df.idProducto.apply(lambda x: x.strip() if x==str else x)
    #saco las cadenas con más de 20 caracteres por considerarse outliers
    df['idProducto']=df.idProducto.apply(lambda x: 0 if type(x)==str and len(x)>20 else x)
    #sacamos carácteres especiales por si los hubiera
    df['idProducto']=df.idProducto.apply(lambda x: normalize( 'NFC', x) if type(x)==str else x)
    #recorto id a 13 valores
    df['idProducto'] = df['idProducto'].apply(lambda x: int(x[-13:]) if(type(x) == str) else x) 
    #elimino los valores que no pueda manipular
    #df['idProducto'] = df['idProducto'].apply(lambda x: 0 if type(x)==str else x)
    return df

In [15]:
Estado_general_tabla(sucursal)

C:\Users\Gise\AppData\Local\Temp\ipykernel_13568\3532138181.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c][i]=s


In [16]:
limpieza_idsucursal (sucursal)

,idSucursal,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
0,7,1,1,Super Mami,Dinosaurio S.A.,Ar-X,Salsipuedes,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,1,10,1,Hipermercado Carrefour,Inc S.A.,Ar-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10,10,1,Hipermercado Carrefour,Inc S.A.,Ar-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,11,10,1,Hipermercado Carrefour,Inc S.A.,Ar-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,112,10,1,Hipermercado Carrefour,Inc S.A.,Ar-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado
...,...,...,...,...,...,...,...,...,...,...,...,...
2328,5277,9,3,Jumbo,Jumbo Retail Argentina S.A.,Ar-U,Comodoro Rivadavia,Yrigoyen Hipolito 0,-45.873300,-67.493500,Jumbo-Comodoro,Hipermercado
2329,5626,9,3,Jumbo,Jumbo Retail Argentina S.A.,Ar-B,General Pacheco,Boulogne Sur Mer 1275,-34.474500,-58.625700,Jumbo Pacheco Novo,Hipermercado
2330,5961,9,3,Jumbo,Jumbo Retail Argentina S.A.,Ar-C,Ciudad Autonoma Buenos Aires,Avenida Santa Fe 4950,-34.577200,-58.430000,Jumbo Av. Santa Fe,Supermercado
2331,628,9,3,Jumbo,Jumbo Retail Argentina S.A.,Ar-B,San Fernando,Avenida Del Libertador Gral San Martin 2271,-34.446900,-58.545700,Jumbo San Fernando,Supermercado


In [17]:
Estado_general_tabla(producto)

Se ha borrado la columna:  categoria1 Por falta de datos
Se ha borrado la columna:  categoria2 Por falta de datos
Se ha borrado la columna:  categoria3 Por falta de datos


In [20]:
limpieza_producto(producto)

,idProducto,marca,nombreProducto,presentacion
0,1663,La Anonima,Radicheta Atada La Anonima,1.0 Un
1,2288,La Anonima,Perejil Atado La Anonima,1.0 Un
2,205870,Sin Marca,Ojo De Bife,1.0 Kg
3,205894,Sin Marca,Milanesa De Peceto Novillito,1.0 Kg
4,205955,Sin Marca,Chiquizuela Novillito,1.0 Kg
...,...,...,...,...
72033,9569753142128,Deli-Sitas,Milhojas Cobertura De Chocolate Blanco Deli-Sitas,500.0 Gr
72034,9795403001143,Mayo,Mini Pizzetas Mayo,12.0 Un
72035,9990385651922,Dana,Te Negro En Hebras Lata Dana,50.0 Gr
72036,9990385651939,Dana,Te Verde En Hebras Lata Dana,50.0 Gr


ahora vemos la de precios

In [21]:
def crear_precio ():
   global precio 
   precio=pd.DataFrame(columns=['año', 'mes', 'día', 'idProducto', 'idSucursal', 'precio', 'comercioId',
      'banderaId', 'banderaDescripcion', 'comercioRazonSocial', 'provincia',
      'localidad', 'direccion', 'lat', 'lng', 'sucursalNombre',
      'sucursalTipo', 'marca', 'nombreProducto', 'presentacion'])
   return precio

In [23]:
#FUNCION PARA LA CARGA INCREMENTAL
def carga_semana (ruta):
    #CARGA DE DATOS Y EXTRACCIÓN DE FECHA:
    #limpio la ruta del atchivo dejando solo el nombre y la extensión
    tupla_archivos=os.path.split(ruta)
    #de la tupla que me devuelve la divido en el nombre que lo voy a usar como suministro de fecha y en extensión
    nombre_tipo_archivo=tupla_archivos[1]
    #la primer parte de la tupla es el nombre por lo que lo guardo en una variable   
    nombre_archivo= os.path.splitext(nombre_tipo_archivo)[0]
    #la segunda parte de la tupla es el tipo de archivo que es que lo uso para saber cómo ingestar
    extension_archivo= os.path.splitext(nombre_tipo_archivo)[-1]
    #detección de la manera de abrirlo en base a la extensión del archivo
    if extension_archivo==".json":
        #desde json
        df=pd.read_json(ruta)
    elif extension_archivo==".csv":
        #desde csv
        try:
            df=pd.read_csv(ruta,sep=",",encoding='utf-16')
        except:
            df=pd.read_json(ruta,sep=",",encoding='auto')
    elif extension_archivo==".txt":
    #desde txt
        df=pd.read_csv(ruta,sep="|",encoding='utf-8')
    elif extension_archivo==".xlsx":
        #desde un excel
        df=pd.read_excel(ruta,header=0)
        
        #Normalizamos
        df['producto_id'] = df['producto_id'].apply(lambda x: int(x[-13:]) if(type(x) == str) else x)
        df['sucursal_id'] = df['sucursal_id'].apply(lambda x: ('{0}-{1}-{2}'.format(int(x.day),int(x.month),int(x.year))) if(type(x) == datetime.datetime) else x)
        df['sucursal_id'] = df['sucursal_id'].apply(lambda x: str(x))
        df['producto_id'] = df['producto_id'].fillna(int(0)).apply(lambda x: str(str(int(float(x))).zfill(13)))
        
    #MODIFICACIÓN DEL DATO:
    #agrego la columna año y semana
    df['año']=int(nombre_archivo[-8:-4])
    df['mes']=int(nombre_archivo[-4:-2])
    df['día']=int(nombre_archivo[-2:len(nombre_archivo)+1])
    #renombro las columnas
    df.rename(columns={ 'producto_id':'idProducto', 'sucursal_id':'idSucursal'},inplace=True)
    #reordeno las columnas  
    df=df.reindex(columns=['año','mes','día','idProducto','idSucursal','precio'])

    return df
    

In [24]:
def limpieza_semana (df=Frame):
    #le hago las limpiezas pertinentes:
    #borramos los valores null debido a que no nos sirve no tener el dato completo
    df.dropna(how='any',inplace= True)
    
    #Borramos duplicados 
    df.drop_duplicates(inplace= True)

    #Saco los espacios
    df.columns = df.columns.str.replace(' ', '')
    
    #Normalización idProducto
    df=limpieza_id_producto(df)

    #Normalización idSucursal:
    #separo los id que tienen - y las guardo en una variable
    ver_id=df['idSucursal'].str.split('-', expand=True)
    #renombro las columnas para poder usarlas
    ver_id.columns=["comercioid",'banderaid',"idSucursal"]
    df['idSucursal']=ver_id['idSucursal']
    
    #Normalización de columna precio
    #Saco los espacios si existen en la columna precio
    df['precio']=df.precio.apply(lambda x: x.strip() if x==str else x)
    #saco las cadenas con más de 20 caracteres por considerarse outliers
    df['precio']=df.precio.apply(lambda x: 0 if type(x)==str and len(x)>20 else x)
    #sacamos carácteres especiales por si los hubiera
    df['precio']=df.precio.apply(lambda x: normalize( 'NFC', x) if type(x)==str else x)
    #elimino los valores que no pueda manipular
    df['precio'] = df['precio'].apply(lambda x: 0 if type(x)==str else x)    
    
    #Cambio el tipo de dato
    df['año'] = df['año'].astype('int32')
    df['mes'] =df['mes'].astype('int32')
    df['día'] = df['día'].astype('int32')
    #df['idProducto'] = df['idProducto'].astype('int32') no llegamos pero lo dejamos para que lo hagan ustedes en el futuro
    df['idSucursal'] = df['idSucursal'].astype('int32')
    df['precio'] = df['precio'].astype('float64')
    
    #combino con la tabla de sucursal y producto y luego actualizo la de precios --> lo convierto en una única tabla!
    df=df.merge(sucursal, on='idSucursal',how='left')
    df=df.merge(producto, on='idProducto',how='left')

    return(df)

In [25]:
def actualizar_cargar_precios (ruta):
    #ruta=input("Inserte la ruta COMPLETA de su archivo de precios")--> para que expandan el proyecto deben modificar un poco esta función pero depende de la necesidad
    #ruta=concat
    df=carga_semana(ruta)
    df=limpieza_semana(df)
    global last_price
    last_price=pd.concat([last_price,df])
    #eliminamos duplicaodos
    last_price.drop_duplicates(inplace=True) 
    return(last_price)   

In [29]:
last_price=pd.DataFrame()

In [30]:
last_price=actualizar_cargar_precios (r'C:\Users\Gise\Desktop\Data Since\LABS\Individuales\Datasets relevamiento precios\precios_semana_20200413.csv')

In [32]:
last_price

,año,mes,día,idProducto,idSucursal,precio,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo,marca,nombreProducto,presentacion
0,2020,4,13,1663,14,29.9,13,1,Cooperativa Obrera Limitada De Consumo Y Vivienda,Cooperativa Obrera Limitada De Consumo Y Vivienda,Ar-B,Pedro Luro,Calle 1 1031,-39.505847,-62.685221,Pedro Luro,Supermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
1,2020,4,13,1663,14,29.9,15,1,Supermercados Dia,Dia Argentina S.A,Ar-B,San Martin,Cl Belgrano 3162,-34.573864,-58.532632,14 - San Martin,Autoservicio,La Anonima,Radicheta Atada La Anonima,1.0 Un
2,2020,4,13,1663,14,29.9,2,1,La Anonima,S.A. Importadora Y Exportadora De La Patagonia,Ar-U,Puerto Madryn,H.Yrigoyen 76,-42.764220,-65.036230,Puerto Madryn,Supermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
3,2020,4,13,1663,14,29.9,21,1,Toledo,Supermercados Toledo S.A.,Ar-B,Mar Del Plata,Av. Jara 3560,-38.002040,-57.585284,Jara,Supermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
4,2020,4,13,1663,14,29.9,6,1,Supermercados Comodin,Alberdi S.A.,Ar-Y,San Salvador De Jujuy,Avda. Corrientes 3141,-24.216700,-65.272455,Comodin 14,Hipermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3179163,2020,4,13,9990385651946,3,198.9,5,1,California Supermercados,California S.A.,Ar-N,Posadas,Av. Lavalle 5495,-27.396928,-55.909301,Local 3,Hipermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr
3179164,2020,4,13,9990385651946,3,198.9,6,1,Supermercados Comodin,Alberdi S.A.,Ar-Y,San Salvador De Jujuy,Dr. Carrillo 371,-24.187378,-65.319964,Comodin 3,Supermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr
3179165,2020,4,13,9990385651946,3,198.9,7,1,Supermercados Cordiez,Cyre S.A.,Ar-X,Las Varillas,Diego Montoya 335,-31.872108,-62.725261,Cordiez 3,Supermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr
3179166,2020,4,13,9990385651946,3,198.9,8,1,Mariano Max,Mariano S.A.,Ar-X,Cordoba,Carlos Gimenez 35,-31.365896,-64.166440,Sucursal 3,Supermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr


Repetimos para el resto de los archivos

In [33]:
last_price=actualizar_cargar_precios(r'C:\Users\Gise\Desktop\Data Since\LABS\Individuales\Datasets relevamiento precios\precios_semana_20200503.json')

In [34]:
last_price

,año,mes,día,idProducto,idSucursal,precio,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo,marca,nombreProducto,presentacion
0,2020,4,13,1663,14,29.9,13.0,1.0,Cooperativa Obrera Limitada De Consumo Y Vivienda,Cooperativa Obrera Limitada De Consumo Y Vivienda,Ar-B,Pedro Luro,Calle 1 1031,-39.505847,-62.685221,Pedro Luro,Supermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
1,2020,4,13,1663,14,29.9,15.0,1.0,Supermercados Dia,Dia Argentina S.A,Ar-B,San Martin,Cl Belgrano 3162,-34.573864,-58.532632,14 - San Martin,Autoservicio,La Anonima,Radicheta Atada La Anonima,1.0 Un
2,2020,4,13,1663,14,29.9,2.0,1.0,La Anonima,S.A. Importadora Y Exportadora De La Patagonia,Ar-U,Puerto Madryn,H.Yrigoyen 76,-42.764220,-65.036230,Puerto Madryn,Supermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
3,2020,4,13,1663,14,29.9,21.0,1.0,Toledo,Supermercados Toledo S.A.,Ar-B,Mar Del Plata,Av. Jara 3560,-38.002040,-57.585284,Jara,Supermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
4,2020,4,13,1663,14,29.9,6.0,1.0,Supermercados Comodin,Alberdi S.A.,Ar-Y,San Salvador De Jujuy,Avda. Corrientes 3141,-24.216700,-65.272455,Comodin 14,Hipermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054982,2020,5,3,9990385651946,5,198.9,45.0,1.0,Almacenes Pampas,Almacenes Pampas S.A.,Ar-B,Berazategui,Autopista Buenos Aires - Mar Del Plata 32,-34.796523,-58.159203,Almacenes Pampas,Supermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr
3054983,2020,5,3,9990385651946,5,198.9,5.0,1.0,California Supermercados,California S.A.,Ar-N,Posadas,Juan B. Alberdi 1095,-27.360006,-55.893729,Local 5,Hipermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr
3054984,2020,5,3,9990385651946,5,198.9,6.0,1.0,Supermercados Comodin,Alberdi S.A.,Ar-Y,San Salvador De Jujuy,Rocha Solorzano 950,-24.200465,-65.289007,Comodin 5,Hipermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr
3054985,2020,5,3,9990385651946,5,198.9,7.0,1.0,Supermercados Cordiez,Cyre S.A.,Ar-X,Villa Dolores,Belgrano 1228,-31.948391,-65.174243,Cordiez 5,Supermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr


In [35]:
last_price=actualizar_cargar_precios(r'C:\Users\Gise\Desktop\Data Since\LABS\Individuales\Datasets relevamiento precios\precios_semana_20200518.txt')

In [36]:
last_price=actualizar_cargar_precios(r'C:\Users\Gise\Desktop\Data Since\LABS\Individuales\Datasets relevamiento precios\precios_semanas_20200419_20200426.xlsx')

In [37]:
last_price

,año,mes,día,idProducto,idSucursal,precio,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo,marca,nombreProducto,presentacion
0,2020,4,13,1663,14,29.9,13.0,1.0,Cooperativa Obrera Limitada De Consumo Y Vivienda,Cooperativa Obrera Limitada De Consumo Y Vivienda,Ar-B,Pedro Luro,Calle 1 1031,-39.505847,-62.685221,Pedro Luro,Supermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
1,2020,4,13,1663,14,29.9,15.0,1.0,Supermercados Dia,Dia Argentina S.A,Ar-B,San Martin,Cl Belgrano 3162,-34.573864,-58.532632,14 - San Martin,Autoservicio,La Anonima,Radicheta Atada La Anonima,1.0 Un
2,2020,4,13,1663,14,29.9,2.0,1.0,La Anonima,S.A. Importadora Y Exportadora De La Patagonia,Ar-U,Puerto Madryn,H.Yrigoyen 76,-42.764220,-65.036230,Puerto Madryn,Supermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
3,2020,4,13,1663,14,29.9,21.0,1.0,Toledo,Supermercados Toledo S.A.,Ar-B,Mar Del Plata,Av. Jara 3560,-38.002040,-57.585284,Jara,Supermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
4,2020,4,13,1663,14,29.9,6.0,1.0,Supermercados Comodin,Alberdi S.A.,Ar-Y,San Salvador De Jujuy,Avda. Corrientes 3141,-24.216700,-65.272455,Comodin 14,Hipermercado,La Anonima,Radicheta Atada La Anonima,1.0 Un
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3177652,2020,4,26,9990385651946,3,1989.0,5.0,1.0,California Supermercados,California S.A.,Ar-N,Posadas,Av. Lavalle 5495,-27.396928,-55.909301,Local 3,Hipermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr
3177653,2020,4,26,9990385651946,3,1989.0,6.0,1.0,Supermercados Comodin,Alberdi S.A.,Ar-Y,San Salvador De Jujuy,Dr. Carrillo 371,-24.187378,-65.319964,Comodin 3,Supermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr
3177654,2020,4,26,9990385651946,3,1989.0,7.0,1.0,Supermercados Cordiez,Cyre S.A.,Ar-X,Las Varillas,Diego Montoya 335,-31.872108,-62.725261,Cordiez 3,Supermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr
3177655,2020,4,26,9990385651946,3,1989.0,8.0,1.0,Mariano Max,Mariano S.A.,Ar-X,Cordoba,Carlos Gimenez 35,-31.365896,-64.166440,Sucursal 3,Supermercado,Dana,Yerba Mate Aromatizada Lata Dana,150.0 Gr


#### Enviamos las tablas a MySQL

In [38]:
from sqlalchemy import create_engine
import pyodbc
import os

In [40]:
#creamos conexión con MySQL
cadena_conexion="mysql+pymysql://root@127.0.0.1:3306/labs_inidividual_01"
conexion=create_engine(cadena_conexion)

In [41]:
#pasamos a mysql la tabla de precios semana 
last_price.to_sql(name="precios",con=conexion,if_exists="replace",index=False)

10949423

In [42]:
#pasamos a mysql la tabla de sucursal
sucursal.to_sql(name="sucursal",con=conexion,if_exists="replace",index=False)

2333

In [43]:
#pasamos a mysql a tabla de producto
producto.to_sql(name="producto",con=conexion,if_exists="replace",index=False)

72038